## Problema da mochila - Algoritmos Genéticos - UFABC - Q1.2024 - Gabriel Meale 11091915

### Configs

In [1]:
import random
from typing import List

In [2]:
ITENS = [
    {"nome": "Barraca", "valor": 150, "peso": 3.5},
    {"nome": "Saco de dormir", "valor": 100, "peso": 2.0},
    {"nome": "Isolante térmico", "valor": 50, "peso": 0.5},
    {"nome": "Colchão inflável", "valor": 80, "peso": 1.0},
    {"nome": "Lanterna", "valor": 30, "peso": 0.2},
    {"nome": "Kit de primeiros socorros", "valor": 20, "peso": 0.5},
    {"nome": "Repelente de insetos", "valor": 15, "peso": 0.1},
    {"nome": "Protetor solar", "valor": 20, "peso": 0.2},
    {"nome": "Canivete", "valor": 10, "peso": 0.1},
    {"nome": "Mapa e bússola", "valor": 25, "peso": 0.3},
    {"nome": "Garrafa de água", "valor": 15, "peso": 1.8},
    {"nome": "Filtro de água", "valor": 50, "peso": 0.5},
    {"nome": "Comida (ração liofilizada)", "valor": 50, "peso": 3.0},
    {"nome": "Fogão de camping", "valor": 70, "peso": 1.5},
    {"nome": "Botijão de gás", "valor": 30, "peso": 1.2},
    {"nome": "Prato, talheres e caneca", "valor": 20, "peso": 0.5},
    {"nome": "Roupas (conjunto)", "valor": 80, "peso": 1.5},
    {"nome": "Calçados (botas)", "valor": 120, "peso": 2.0},
    {"nome": "Toalha", "valor": 20, "peso": 0.5},
    {"nome": "Kit de higiene pessoal", "valor": 30, "peso": 0.5}
]

CAPACIDADE_MOCHILA = 15.0

TORNEIO = "torneio"

### Classes auxiliares

In [3]:
class Individuo:
    '''Classe que responsável por operações de indivíduo'''

    def __init__(self):
        self.cromossomo = self.criar_individuo()

    def criar_individuo(self):
        """Cria um indivíduo (cromossomo) aleatório."""
        return [random.randint(0, 1) for _ in range(len(ITENS))]


In [4]:
class AlgoritmoGenetico:
    '''
        Classe que implementa o algoritmo genético
    '''

    def __init__(
            self, tamanho_populacao : int = 100,
            max_geracoes : int = 100,
            taxa_mutacao : float = 0.1,
            mutacao_flag : bool = True,
            crossover_flag : bool = True,
            tipo_selecao = TORNEIO,
            torneio_size : int = 3
        ):
        self.tamanho_populacao = tamanho_populacao
        self.max_geracoes = max_geracoes
        self.taxa_mutacao = taxa_mutacao
        self.mutacao_flag = mutacao_flag
        self.crossover_flag = crossover_flag
        self.populacao = []
        self.tipo_selecao = tipo_selecao
        self.torneio_size = torneio_size

    def criar_populacao(self):
        '''
            Função que cria a população de indivíduos
        '''
        self.populacao = []

        for _ in range(self.tamanho_populacao):
            self.populacao.append(Individuo())

        return self.populacao

    def mutacao(self, individuo: Individuo):
        '''
            Realiza a mutação em um indivíduo trocando aleatoriamente alguns bits.
        '''
        for item in enumerate(individuo.cromossomo):
            if random.random() < self.taxa_mutacao:
                individuo.cromossomo[item[0]] = 1 - individuo.cromossomo[item[0]]

        return individuo

    def crossover_binario(self, pai1: Individuo, pai2: Individuo):
        '''
            Realiza o crossover de ponto único entre dois pais para gerar dois filhos.
        '''
        ponto_corte = random.randint(0, len(pai1.cromossomo) - 1)

        filho1 = Individuo()
        filho2 = Individuo()

        filho1.cromossomo = pai1.cromossomo[:ponto_corte] + pai2.cromossomo[ponto_corte:]
        filho2.cromossomo = pai2.cromossomo[:ponto_corte] + pai1.cromossomo[ponto_corte:]

        return filho1, filho2

    def selecao_torneio(self, populacao: List[Individuo]) -> list:
        '''
            Seleciona os pais usando o método de torneio.
        '''
        pais = []
        for _ in range(2):  # Dois pais para o crossover
            participantes = random.sample(populacao, self.torneio_size)
            fitness = [self.calcular_fitness(participante) for participante in participantes]
            index_melhor_participante = fitness.index(max(fitness))
            melhor_participante = participantes[index_melhor_participante]
            pais.append(melhor_participante)

        return pais

    def calcular_fitness(self, individuo: Individuo):
        '''
            Função que calcula fitness de cada indivíduo
        '''

        valor_total = 0
        peso_total = 0

        for item in enumerate(ITENS):
            if individuo.cromossomo[item[0]] == 1:
                valor_total += ITENS[item[0]]["valor"]
                peso_total += ITENS[item[0]]["peso"]

        if peso_total > CAPACIDADE_MOCHILA:
            return 0 # Penaliza soluções que excedem a capacidade

        return valor_total

    def process(self):
        '''
            Função principal de processamento do algoritmo genetico
        '''

        self.criar_populacao()

        for geracao in range(self.max_geracoes):

            if self.tipo_selecao == TORNEIO:
                pais = self.selecao_torneio(self.populacao)
            else:
                raise NotImplementedError

            nova_populacao = []
            for _ in range(self.tamanho_populacao // 2):
                pai1 = pais[0]
                pai2 = pais[1]
                filho1, filho2 = self.crossover_binario(pai1, pai2)
                filho1 = self.mutacao(filho1)
                filho2 = self.mutacao(filho2)
                nova_populacao.extend([filho1, filho2])

            self.populacao = nova_populacao

            aptidoes = [self.calcular_fitness(individuo) for individuo in self.populacao]
            melhor_aptidao = max(aptidoes)
            melhor_individuo = self.populacao[aptidoes.index(melhor_aptidao)]

            if geracao % 10 == 0:
                print(f"Geração {geracao}: Melhor valor = R${melhor_aptidao}")

        return melhor_individuo, melhor_aptidao


### Execução do código

In [5]:
algoritmo_genetico = AlgoritmoGenetico()
melhor_solucao, melhor_valor = algoritmo_genetico.process()

# double check de fitness para a melhor solução
if algoritmo_genetico.calcular_fitness(individuo=melhor_solucao) == melhor_valor:
    print(f"\nMelhor solução encontrada: {melhor_solucao.cromossomo}")
    print(f"Valor da melhor solução: R${melhor_valor}")

Geração 0: Melhor valor = R$700
Geração 10: Melhor valor = R$800
Geração 20: Melhor valor = R$720
Geração 30: Melhor valor = R$870
Geração 40: Melhor valor = R$870
Geração 50: Melhor valor = R$870
Geração 60: Melhor valor = R$725
Geração 70: Melhor valor = R$675
Geração 80: Melhor valor = R$755
Geração 90: Melhor valor = R$835

Melhor solução encontrada: [1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0]
Valor da melhor solução: R$805


### Ajuste hiperparametros e outros testes

In [12]:
tamanho_populacao = 10
max_geracoes = 1
taxa_mutacao = 0.1
torneio_size = 3
melhor_valor_geral = 0
melhor_cromossomo_geral = []

In [ ]:
for i in range(tamanho_populacao, 200):
    for j in range(max_geracoes, 200):
        taxa_mutacao = 0.1
        while taxa_mutacao <= 1.0:
            for k in range(torneio_size, 10):
                algoritmo_genetico = AlgoritmoGenetico(tamanho_populacao=i, max_geracoes=j, taxa_mutacao=taxa_mutacao, torneio_size=k)
                melhor_solucao, melhor_valor = algoritmo_genetico.process()
                if (melhor_valor >= melhor_valor_geral) and (algoritmo_genetico.calcular_fitness(individuo=melhor_solucao) == melhor_valor):
                    melhor_valor_geral = melhor_valor
                    melhor_cromossomo_geral = melhor_solucao
                    print("--------------------------------------------\n")
                    print(f"\nMelhor solução encontrada: {melhor_solucao.cromossomo}")
                    print(f"Valor da melhor solução: R${melhor_valor}")
                    print(f"tamanho_populacao: {tamanho_populacao}")
                    print(f"max_geracoes: {max_geracoes}")
                    print(f"taxa_mutacao: {taxa_mutacao}")
                    print(f"torneio_size: {torneio_size}")
                    print("--------------------------------------------\n")
            taxa_mutacao += 0.1

### Teste final com hiperparâmetros ajustados

In [32]:
tamanho_populacao = 300
max_geracoes = 300
taxa_mutacao = 0.25
torneio_size = 3

In [33]:
algoritmo_genetico = AlgoritmoGenetico(tamanho_populacao=tamanho_populacao, max_geracoes=max_geracoes, taxa_mutacao=taxa_mutacao, torneio_size=torneio_size)
melhor_solucao, melhor_valor = algoritmo_genetico.process()

if algoritmo_genetico.calcular_fitness(individuo=melhor_solucao) == melhor_valor:
    print(f"\nMelhor solução encontrada: {melhor_solucao.cromossomo}")
    print(f"Valor da melhor solução: R${melhor_valor}")

Geração 0: Melhor valor = R$740
Geração 10: Melhor valor = R$780
Geração 20: Melhor valor = R$740
Geração 30: Melhor valor = R$775
Geração 40: Melhor valor = R$790
Geração 50: Melhor valor = R$765
Geração 60: Melhor valor = R$790
Geração 70: Melhor valor = R$765
Geração 80: Melhor valor = R$740
Geração 90: Melhor valor = R$755
Geração 100: Melhor valor = R$805
Geração 110: Melhor valor = R$825
Geração 120: Melhor valor = R$805
Geração 130: Melhor valor = R$830
Geração 140: Melhor valor = R$775
Geração 150: Melhor valor = R$815
Geração 160: Melhor valor = R$765
Geração 170: Melhor valor = R$770
Geração 180: Melhor valor = R$735
Geração 190: Melhor valor = R$755
Geração 200: Melhor valor = R$805
Geração 210: Melhor valor = R$750
Geração 220: Melhor valor = R$810
Geração 230: Melhor valor = R$800
Geração 240: Melhor valor = R$780
Geração 250: Melhor valor = R$800
Geração 260: Melhor valor = R$805
Geração 270: Melhor valor = R$815
Geração 280: Melhor valor = R$785
Geração 290: Melhor valor